# 모델링 구현

## word2vec을 활용한 모델 구현

- 각 단어에 대해 word2vec으로 벡터화
- 전처리된 데이터를 불러온 후 각 단어들의 리스트로 나눠야 함

In [78]:
import os
import pandas as pd
import numpy as np

In [79]:
DATA_IN_PATH = './data_in/'
TRAIN_CLEAN_DATA = 'train_clean.csv'
train_data = pd.read_csv(DATA_IN_PATH + TRAIN_CLEAN_DATA)

reviews = list(train_data['review'])
sentiments = list(train_data['sentiment'])

sentences = []
for review in reviews:
    sentences.append(review.split())

- 전처리한 데이터의 경우 리뷰가 하나의 문자열로 구성되어 있음
- word2vec를 사용하기 위해서는 입력값을 단어로 구분된 리스트로 만들어야함

### word2vec 벡터화

In [80]:
# word2vec 모델의 하이퍼파라미터 설정
# 학습 시 필요한 하이퍼파라미터
num_features = 300
min_word_count = 40
num_workers = 4
context = 10
downsampling = 1e-3

- num_features
    - 워드 벡터 특징값 수
    - 각 단어에 대해 임베딩된 벡터의 차원을 정한다.

- min_word_count
    - 단어에 대한 최소 빈도 수
    - 모델에 의미 있는 단어를 가지고 학습하기 위해 적은 빈도 수의 단어들은 학습하지 않는다.
    
- num_workers
    - 프로세스 개수
    - 모델 학습 시 학습을 위한 프로세스 개수를 지정한다.

- context
    - 컨텍스트 윈도우 크기
    - word2vec을 수행하기 위한 컨텍스트 윈도우 크기를 지정한다.

- downsampling
    - 다운 샘플링 비율
    - word2vec 학습을 수행할 때 빠른 학습을 위해 정답 단어 라벨에 대한 다운샘플링 비율을 지정한다. 보통 0.001이 좋은 성능을 낸다고 한다.

In [81]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

- 로깅을 할 때 format을 위와 같이 지정하고 로그 수준은 INFO에 맞추면 word2vec의 학습 과정에서 로그 메세지를 양식에 맞게 INFO 수준으로 보여준다

- word2vec 모듈에 있는 Word2Vec 객체를 생성해서 실행
- 학습하고 생성된 객체는 model 변수에 할당

In [82]:
from gensim.models import word2vec

print("Training model.....")
model = word2vec.Word2Vec(
    sentences,
    workers = num_workers,
    size = num_features,
    min_count = min_word_count,
    window = context,
    sample = downsampling)

2020-02-15 14:31:30,764 : INFO : collecting all words and their counts
2020-02-15 14:31:30,767 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
Training model.....
2020-02-15 14:31:31,748 : INFO : PROGRESS: at sentence #10000, processed 1205223 words, keeping 51374 word types
2020-02-15 14:31:32,675 : INFO : PROGRESS: at sentence #20000, processed 2396605 words, keeping 67660 word types
2020-02-15 14:31:33,124 : INFO : collected 74065 word types from a corpus of 2988089 raw words and 25000 sentences
2020-02-15 14:31:33,126 : INFO : Loading a fresh vocabulary
2020-02-15 14:31:33,281 : INFO : effective_min_count=40 retains 8160 unique words (11% of original 74065, drops 65905)
2020-02-15 14:31:33,282 : INFO : effective_min_count=40 leaves 2627273 word corpus (87% of original 2988089, drops 360816)
2020-02-15 14:31:33,396 : INFO : deleting the raw counts dictionary of 74065 items
2020-02-15 14:31:33,402 : INFO : sample=0.001 downsamples 30 most-common words
2020-

- 모델의 하이퍼파라미터를 설정한 내용을 모델 이름에 담는다면 나중에 참고하기에 좋을 것이다.
- 모델을 저장하면 Word2Vec.load()를 통해 모델을 다시 사용할 수 있다.

In [83]:
# model 저장하기
# model_name = "300features_10minwords_10context"
# model.save(model_name)

### 선형 회귀 모델 학습

- 입력값 만드는 방법
    - 문장에 있는 모든 단어의 벡터값에 대해 평균을 내서 리뷰 하나당 하나의 벡터로 만드는 방법

In [84]:
# 하나의 리뷰에 대해 전체 단어의 평균값을 계산하는 함수
def get_features(words, model, num_features):
    # 출력 벡터 초기화
    feature_vector = np.zeros((num_features), dtype=np.float32)

    num_words = 0

    # 어휘사전 준비
    index2word_set = set(model.wv.index2word)

    for w in words:
        if w in index2word_set:
            num_words += 1
            # 사전에 해당하는 단어에 대해 단어 벡터를 더함
            feature_vector = np.add(feature_vector, model[w])

    # 문장의 단어 수만큼 나누어 단어 벡터의 평균값을 문장 벡터로 함
    feature_vector = np.divide(feature_vector, num_words)
    return feature_vector

- words
    - 단어의 모음인 하나의 리뷰가 들어간다.

- model
    - word2vec 모델을 넣는 곳이며, 우리가 학습한 word2vec 모델이 들어간다.

- num_features
    - word2vec으로 임베딩할때 정했던 벡터의 차원 수를 뜻한다.

- 하나의 벡터를 만드는 과정에서 속도를 빠르게 하기 위해 np.zeros를 사용해 미리 모두 0의 값을 가지는 벡터를 만듬
- 문장의 단어가 해당 모델 단어사전에 속하는지 보기 위해 model.wv.index2word를 set 객체로 생성해서 index2word_set 변수에 할당함
- 반복문을 통해 리뷰를 구성하는 단어에 대해 임베딩된 벡터가 있는 단어 벡터의 합을 구함
- 사용한 단어의 전체 개수로 나눔으로써 평균 벡터의 값을 구

In [85]:
# 전체 리뷰에 대해 각 리뷰의 평균 벡터를 구하는 함수
def get_dataset(reviews, model, num_features):
    dataset = list()

    for s in reviews:
        dataset.append(get_features(s, model, num_features))

    reviewFeatureVecs = np.stack(dataset)

    return reviewFeatureVecs

- reviews
    - 학습 데이터인 전체 리뷰 데이터를 입력하는 인자

- model
    - word2vec 모델을 입력하는 인자
    - 위 함수와 같이 앞에서 학습시킨 모델을 넣는다.

- num_features
    - word2vec으로 임베딩할때 정했던 벡터의 차원 수를 뜻한다.

- 속도 향상을 위해 전체 리뷰에 대한 평균 벡터를 담을 0 으로 채워진 넘파이 배열을 미리 만든다.
- 배열은 2차원으로 만드는데 배열의 행에는 각 문장에 대한 길이를 입력하면 되고 열에는 평균 벡터의 차원 수, 즉 크기를 입력한다.
- 각 리뷰에 대해 반복문을 돌면서 각 리뷰에 대해 특징 값을 만든다.

In [86]:
# 전체 리뷰 데이터를 위 함수를 사용해 실제 학습에 사용될 입력값 만들기
test_data_vecs = get_dataset(sentences, model, num_features)

In [87]:
test_data_vecs

array([[-0.05141454, -0.09214196, -0.10656108, ..., -0.25861964,
         0.11964124,  0.15288605],
       [ 0.21623717, -0.07709181, -0.06491359, ..., -0.13432683,
         0.15685375, -0.25512555],
       [-0.12524039, -0.01853671,  0.10024042, ..., -0.17027494,
         0.17265838,  0.00813573],
       ...,
       [ 0.30068046, -0.17251717, -0.15885326, ..., -0.17351472,
         0.08153304,  0.3920877 ],
       [ 0.1485902 ,  0.05575932, -0.0201204 , ..., -0.21828656,
         0.08524875, -0.00382755],
       [-0.03531034,  0.15706989,  0.06043947, ..., -0.25309756,
         0.1035813 ,  0.11736225]], dtype=float32)

### 학습 데이터셋과 검증 데이터셋 분리 

In [88]:
from sklearn.model_selection import train_test_split

X = test_data_vecs
y = np.array(sentiments)

RANDOM_SEED = 42
TEST_SPLIT = 0.2

X_train, X_eval, y_train, y_eval = train_test_split(X, y, test_size=TEST_SPLIT, random_state=RANDOM_SEED)

### 모델 선언 및 학습
- 로지스틱 모델 사용

In [89]:
from sklearn.linear_model import LogisticRegression

lgs = LogisticRegression(class_weight='balanced')
lgs.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight='balanced', dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [90]:
X_train

array([[ 0.07283596, -0.10686829, -0.05979674, ..., -0.3413663 ,
         0.14170878,  0.18238851],
       [ 0.30186263, -0.00209423, -0.21597567, ..., -0.50307095,
        -0.14702682,  0.3526893 ],
       [ 0.35746956, -0.20288137, -0.19624706, ..., -0.19885197,
        -0.03129605,  0.05283298],
       ...,
       [ 0.0798316 , -0.06252076,  0.04627676, ...,  0.00895521,
         0.11041459, -0.1396915 ],
       [-0.00516012,  0.12322056, -0.11593889, ..., -0.22621252,
         0.14311898,  0.00674704],
       [ 0.40598226, -0.17368406, -0.21998726, ..., -0.241794  ,
        -0.05144045,  0.04521733]], dtype=float32)

In [91]:
lgs.predict(X_train)

array([0, 0, 0, ..., 1, 1, 0])

### 검증 데이터셋을 이용한 성능 평가

In [92]:
print(f"Accuracy: {lgs.score(X_eval, y_eval)}")

Accuracy: 0.8622


### Kaggle 데이터 재출

In [93]:
TEST_CLEAN_DATA = 'test_clean.csv'

test_data = pd.read_csv(DATA_IN_PATH + TEST_CLEAN_DATA)

test_review = list(test_data['review'])

In [94]:
test_data.head()

,review,id
0,naturally film main themes mortality nostalgia...,"""12311_10"""
1,movie disaster within disaster film full great...,"""8348_2"""
2,movie kids saw tonight child loved one point k...,"""5828_4"""
3,afraid dark left impression several different ...,"""7186_2"""
4,accurate depiction small time mob life filmed ...,"""12128_7"""


In [95]:
# 평가 데이터를 각 단어의 리스트로 만들기
test_sentences = list()
for review in test_review:
    test_sentences.append(review.split())

In [96]:
test_sentences

eheader',
  'certainly',
  'something',
  'like',
  'never',
  'seen',
  'movies',
  'something',
  'seen',
  'shall',
  'probably',
  'never',
  'forget',
  'even',
  'wish',
  'could',
  'wine',
  'drinking',
  'penis',
  'roped',
  'seen',
  'shame',
  'forever',
  'things',
  'strike',
  'director',
  'porn',
  'magnate',
  'fantasies',
  'sprung',
  'sick',
  'imaginations',
  'proved',
  'record',
  'acting',
  'pretty',
  'good',
  'actually',
  'think',
  'mcdowell',
  'weakest',
  'link',
  'mirren',
  'always',
  'something',
  'behold',
  'youthful',
  'years',
  'formidable',
  'beautiful',
  'portrayals',
  'nerva',
  'tiberius',
  'done',
  'well',
  'respected',
  'actors',
  'film',
  'could',
  'done',
  'much',
  'better',
  'story',
  'telling',
  'failure',
  'way',
  'whole',
  'layer',
  'shame',
  'thing',
  'example',
  'could',
  'given',
  'opinion',
  'caligula',
  'went',
  'mad',
  'shown',
  'mad',
  'paranoid',
  'illness',
  'fever',
  'broke',
  'mental

In [97]:
# 이전에 학습 시킨 모델을 사용해 각 단어들을 벡터로 만들어 각 리뷰에 대한 특징값 만들기
test_data_vecs = get_dataset(test_sentences, model, num_features)

In [98]:
test_data_vecs

array([[ 0.03479357,  0.02736662, -0.05787911, ..., -0.11714542,
         0.10625131,  0.04002199],
       [ 0.26386493, -0.13887046, -0.07351666, ..., -0.3478534 ,
         0.13579936,  0.2176599 ],
       [ 0.13160817, -0.03928975, -0.1781028 , ..., -0.27594152,
         0.22160776,  0.05922965],
       ...,
       [ 0.18991743,  0.00637124, -0.09258428, ..., -0.2783701 ,
        -0.01602164,  0.22869526],
       [-0.28402674,  0.20114522,  0.01796369, ..., -0.10727764,
         0.03736885,  0.0606447 ],
       [-0.0604426 , -0.23065265, -0.12924203, ..., -0.20908059,
         0.3107148 ,  0.11297662]], dtype=float32)

In [99]:
lgs.predict(test_data_vecs)

array([1, 0, 1, ..., 0, 1, 1])

- 평가 데이터에 대해 각 리뷰를 특징 벡터로 만들었으면 이제 로지스틱 모델에 적용
- 결과를 csv 파일로 저장

In [100]:
DATA_OUT_PATH = './data_out/'

test_predicted = lgs.predict(test_data_vecs)

if not os.path.exists(DATA_OUT_PATH):
    os.makedirs(DATA_OUT_PATH)

ids = list(test_data['id'])
answer_dataset = pd.DataFrame({
    'id': ids,
    'sentiment': test_predicted
})
answer_dataset.to_csv(DATA_OUT_PATH + 'lgs_answer.csv', index=False, quoting=3, mode='w')